In [1]:
import pyugo_db

In [8]:
import urllib.parse as urlparse

# 概要
yugipedia.com/wiki にアクセスし、カードテキストを取得する流れを検証。

ルートURL: https://yugipedia.com/wiki

## 内容

1. URLを組み立てて、カードページにアクセス
3. カードページからテキスト部分を取得

## 注意
アクセスするときは、pyugo_db.crawlers.base.YGOCrawlerBaseを必ず使うこと

In [3]:
from pyugo_db.settings import PYUGO_DB_LIST_CRAWLING_ROOT_URL

In [4]:
from pyugo_db.crawlers.base import YGOCrawlerBase

In [5]:
crawler = YGOCrawlerBase()

In [184]:
def skip_rb(root_elm):
    text = ''
    for elm in root_elm.children:
        if isinstance(elm, Tag) and (elm.name == 'rb'):
            text += ''.join([e for e in elm.children if isinstance(e, str)])
        elif isinstance(elm, str):
            text += elm
    return text

### 1. カードページへのアクセス

In [7]:
card_name = 'オベリスクの巨神兵'
card_no = 'MVPY-JP001'

In [16]:
search_html = crawler.get_html('https://yugipedia.com/wiki' + '/' + card_no)

In [18]:
card_attributes_elm = search_html.find('div', class_='card-table')

In [197]:
card_name_elm = card_attributes_elm.find('span', class_='nowrap', attrs={'lang': 'ja'})
japanese_card_name = ''.join([v if isinstance(v, str) else skip_rb(v) for v in card_name_elm.children])
if card_attributes_elm.find('span', class_='nowrap', attrs={'lang': 'ja-Hrkt'}) is None:
    japanese_card_kana = japanese_card_name
else:
    japanese_card_kana = card_attributes_elm.find('span', class_='nowrap', attrs={'lang': 'ja-Hrkt'}).text

In [198]:
japanese_card_name, japanese_card_kana

('オベリスクの巨神兵', 'オベリスクのきょしんへい')

In [100]:
info_table = card_attributes_elm.find('div', class_='infocolumn').find('table')

In [102]:
info_rows = info_table.find_all('tr')

In [142]:
info_label_values = {}
for info_row in info_rows:
    if info_row.find('th') is None:
        continue
    
    info_labels = [e.text for e in info_row.find('th').find_all('a') if e.text != '']
    
    if len(info_labels) == 0:
        continue
    
    info_label = info_labels[0]
    info_values = [e.text for e in info_row.find('td').find_all('a') if e.text != '']
    
    info_label_values[info_label] = info_values

In [143]:
# 種類
card_type = None

# 属性
card_attr = None
# 種族
type_ = None

# monster type
## 効果
has_effect = False
## ペンデュラム
is_pendulum = False

# monster card types
## 融合
is_fusion = False
## 儀式
is_ritual = False
## シンクロ
is_synchro = False
## エクシーズ
is_xyz = False
## リンク
is_link = False

# monster subtypes
## チューナー
is_tuner = False
## トゥーン
is_toon = False
## ユニオン
is_union = False
## スピリット
is_sprit = False
## デュアル
is_gemini = False

# レベル
level = None
# ランク
rank = None
# リンクマーカー
link_arrows = None
# 攻撃力
attack = None
# 守備力
defense = None
# リンク
link = None

# property
## 通常魔法
is_normal_spell = False
## 永続魔法
is_continuous_spell = False
## フィールド魔法
is_field_spell = False
## 速攻魔法
is_quick_play_spell = False
## 装備魔法
is_equip_spell = False
## 儀式魔法
is_ritual_spell = False
## 通常トラップ
is_normal_trap = False
## 永続トラップ
is_continuous_trap = False
## 通常トラップ
is_counter_trap = False

# パスワード
password = None
# ステータス
status = None

In [144]:
def get_and_drop(dic, k):
    v = dic[k]
    del dic[k]
    return v

def get_and_drop_assert1(dic, k):
    l = get_and_drop(dic, k)
    assert len(l) == 1
    return l[0]

def get_and_drop_nullable(dic, k):
    l = get_and_drop(dic, k)
    if len(l) == 0:
        return '-'
    else:
        assert len(l) == 1
        return l[0]

def check_and_drop(l: list, v):
    result = v in l
    if result:
        l.remove(v)
    return result

In [145]:
dic = dict(info_label_values)
card_type = get_and_drop_assert1(dic, 'Card type')

if card_type == 'Spell':
    prop = get_and_drop_assert1(dic, 'Property')
    if prop == 'Normal':
        is_normal_spell = True
    elif prop == 'Continuous':
        is_continuous_spell = True
    elif prop == 'Field':
        is_field_spell = True
    elif prop == 'Equip':
        is_equip_spell = True
    elif prop == 'Quick-Play':
        is_quick_play_spell = True
    elif prop == 'Ritual':
        is_ritual_spell = True
    else:
        assert False,  (card_type, prop)
elif card_type == 'Trap':
    prop = get_and_drop_assert1(dic, 'Property')
    if prop == 'Normal':
        is_normal_trap = True
    elif prop == 'Continuous':
        is_continuous_trap = True
    elif prop == 'Counter':
        is_field_trap = True
    else:
        assert False, (card_type, prop)
elif card_type == 'Monster':
    types = get_and_drop(dic, 'Types')
    type_ = types.pop(0)
    card_attr = get_and_drop_assert1(dic, 'Attribute')
    if check_and_drop(types, 'Effect'):
        has_effect = True
    if check_and_drop(types, 'Pendulum'):
        is_pendulum = True
    if check_and_drop(types, 'Fusion'):
        is_fusion = True
    if check_and_drop(types, 'Ritual'):
        is_ritual = True
    if check_and_drop(types, 'Synchro'):
        is_synchro = True
    if check_and_drop(types, 'Xyz'):
        is_xyz = True
    if check_and_drop(types, 'Link'):
        is_link = True
    if check_and_drop(types, 'Tuner'):
        is_tuner = True
    if check_and_drop(types, 'Union'):
        is_union = True
    if check_and_drop(types, 'Toon'):
        is_toon = True
    if check_and_drop(types, 'Spirit'):
        is_spirit = True
    if check_and_drop(types, 'Gemini'):
        is_gemini = True
    assert len(types) == 0, types
    
    if is_link:
        link_arrows = ','.join(get_and_drop(dic, 'Link Arrows'))
    elif is_xyz:
        rank = get_and_drop_assert1(dic, 'Rank')
    else:
        level = get_and_drop_assert1(dic, 'Level')
    
    if is_link:
        attack, link = get_and_drop(dic, 'ATK')
    else:
        attack, defense = get_and_drop(dic, 'ATK')
    password = get_and_drop_nullable(dic, 'Password')
    status_all: list = get_and_drop(dic, 'Status')
    try:
        ocg_idx = status_all.index('OCG')
        status = status_all[ocg_idx - 1]
    except:
        pass

    assert len(dic) == 0, dic

'Divine-Beast'

In [188]:
name_and_text = list(search_html.find('th', text='Japanese').parent.find_all('td'))

assert len(name_and_text) in (2, 3)
card_text = ''.join([v if isinstance(v, str) else skip_rb(v) for v in name_and_text[1].children])
pendulum_text = None
if len(name_and_text) == 3:
    assert is_pendulum
    pendulum_text = ''.join([v if isinstance(v, str) else skip_rb(v) for v in name_and_text[2].children])

In [189]:
card_text

'このカードを通常召喚する場合、３体をリリースして召喚しなければならない。①：このカードの召喚は無効化されない。②：このカードの召喚成功時には、魔法・罠・モンスターの効果は発動できない。③：このカードは効果の対象にならない。④：自分フィールドのモンスター２体をリリースして発動できる。相手フィールドのモンスターを全て破壊する。この効果を発動するターン、このカードは攻撃宣言できない。⑤：このカードが特殊召喚されている場合、エンドフェイズに発動する。このカードを墓地へ送る。'